# Save Images

This is a demo of saving the [Extreme Heat Forecast](https://data.spatiafi.com/dataset/35) dataset.  We start by loading our App Credentials, and then using the [crop](https://docs.spatiafi.com/api/#operation/geojson_crop_api_image_crop_post) endpoint to fetch a part of the dataset cropped to an area of interest.

---

Install `spatiafi` (if not already installed):

In [ ]:
%pip install --upgrade spatiafi

In [ ]:
from io import BytesIO

import matplotlib.pyplot as plt
import spatiafi
from PIL import Image

### Load SpatiaFi App Credentials from file and create Authenticated `session`

We load previously generated App Credentials.

**If you have not generated App Credentials, please do so first by running the [App Authentication](https://docs.spatiafi.com/tutorials/3-manual-app-authentication/) notebook**

In [ ]:
session = spatiafi.get_session()
session

## Crop Dataset to AOI

Now that we have authenticated to the SpatiaFi API, we can use the `/api/image/crop` endpoint to download a specific Area of Interest (AOI).

In this case, the GeoJSON payload data was found by manually selecting an area using [geojson.io](geojson.io).  We could also generate these AOIs or read them from an existing file.

In [ ]:
url = "https://api.spatiafi.com/api/image/crop"

params = {
    "format": "png",
    "item_id": "ce-extreme-heat-risk-current-global-v1.0",
    "bidx": "1",
    "rescale": "0,4",  # this dataset has a range of values between 0-4, so we rescale the gray values to better see the image.
}

# North Saudi Arabia
payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.76491069149293, 32.29986592075784],
                [31.76491069149293, 23.092931581000542],
                [51.58140483659537, 23.092931581000542],
                [51.58140483659537, 32.29986592075784],
                [31.76491069149293, 32.29986592075784],
            ]
        ],
        "type": "Polygon",
    },
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)
response.raise_for_status()

### Open and save the generated image

Using [PIL](https://pillow.readthedocs.io/en/stable/) and matplotlib, we can quickly view the image in Jupyter before saving it to disk.

In [ ]:
# Open the image file from the response content
img = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(img)
plt.show()

## Save the Image to disk

In [ ]:
# Save image to disk
with open("output.png", "wb") as f:
    for chunk in response.iter_content(1024):
        f.write(chunk)